<a href="https://colab.research.google.com/github/Th-Three/datascience_projects/blob/main/python%20project/Modeling%20Car%20Insurance%20Claim%20Outcomes/%5BDatacamp_Project%5DModeling_Car_Insurance_Claim_Outcomes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Insurance companies invest a lot of [time and money](https://www.accenture.com/_acnmedia/pdf-84/accenture-machine-leaning-insurance.pdf) into optimizing their pricing and accurately estimating the likelihood that customers will make a claim. In many countries insurance it is a legal requirement to have car insurance in order to drive a vehicle on public roads, so the market is very large!

Knowing all of this, On the Road car insurance have requested your services in building a model to predict whether a customer will make a claim on their insurance during the policy period. As they have very little expertise and infrastructure for deploying and monitoring machine learning models, they've asked you to identify the single feature that results in the best performing model, as measured by accuracy, so they can start with a simple model in production.

They have supplied you with their customer data as a csv file called `car_insurance.csv`, along with a table detailing the column names and descriptions below.



## The dataset

| Column | Description |
|--------|-------------|
| `id` | Unique client identifier |
| `age` | Client's age: <br> <ul><li>`0`: 16-25</li><li>`1`: 26-39</li><li>`2`: 40-64</li><li>`3`: 65+</li></ul> |
| `gender` | Client's gender: <br> <ul><li>`0`: Female</li><li>`1`: Male</li></ul> |
| `driving_experience` | Years the client has been driving: <br> <ul><li>`0`: 0-9</li><li>`1`: 10-19</li><li>`2`: 20-29</li><li>`3`: 30+</li></ul> |
| `education` | Client's level of education: <br> <ul><li>`0`: No education</li><li>`1`: High school</li><li>`2`: University</li></ul> |
| `income` | Client's income level: <br> <ul><li>`0`: Poverty</li><li>`1`: Working class</li><li>`2`: Middle class</li><li>`3`: Upper class</li></ul> |
| `credit_score` | Client's credit score (between zero and one) |
| `vehicle_ownership` | Client's vehicle ownership status: <br><ul><li>`0`: Does not own their vehilce (paying off finance)</li><li>`1`: Owns their vehicle</li></ul> |
| `vehcile_year` | Year of vehicle registration: <br><ul><li>`0`: Before 2015</li><li>`1`: 2015 or later</li></ul> |
| `married` | Client's marital status: <br><ul><li>`0`: Not married</li><li>`1`: Married</li></ul> |
| `children` | Client's number of children |
| `postal_code` | Client's postal code |
| `annual_mileage` | Number of miles driven by the client each year |
| `vehicle_type` | Type of car: <br> <ul><li>`0`: Sedan</li><li>`1`: Sports car</li></ul> |
| `speeding_violations` | Total number of speeding violations received by the client |
| `duis` | Number of times the client has been caught driving under the influence of alcohol |
| `past_accidents` | Total number of previous accidents the client has been involved in |
| `outcome` | Whether the client made a claim on their car insurance (response variable): <br><ul><li>`0`: No claim</li><li>`1`: Made a claim</li></ul> |

In [1]:
# Import required modules
import pandas as pd
import numpy as np
from statsmodels.formula.api import logit

# Start coding!

##Task
- Identify the single feature of the data that is the best predictor of whether a customer will put in a claim (the `"outcome"` column), excluding the `"id"` column.
- Store as a DataFrame called `best_feature_df`, containing columns named `"best_feature"` and `"best_accuracy"` with the name of the feature with the highest accuracy, and the respective accuracy score.

In [2]:
#Read the dataset
car_insurance = pd.read_csv('car_insurance.csv')

In [3]:
#Explor the dataset
print('head:\n')
print(car_insurance.head())
print('info:\n')
print(car_insurance.info())
print('describe:\n')
print(car_insurance.describe())
print('Find missing values:')
print(car_insurance.isnull().sum())

head:

       id  age  gender driving_experience    education         income  \
0  569520    3       0               0-9y  high school    upper class   
1  750365    0       1               0-9y         none        poverty   
2  199901    0       0               0-9y  high school  working class   
3  478866    0       1               0-9y   university  working class   
4  731664    1       1             10-19y         none  working class   

   credit_score  vehicle_ownership vehicle_year  married  children  \
0      0.629027                1.0   after 2015      0.0       1.0   
1      0.357757                0.0  before 2015      0.0       0.0   
2      0.493146                1.0  before 2015      0.0       0.0   
3      0.206013                1.0  before 2015      0.0       1.0   
4      0.388366                1.0  before 2015      0.0       0.0   

   postal_code  annual_mileage vehicle_type  speeding_violations  duis  \
0        10238         12000.0        sedan                

In [4]:
#decide what to do with missing values
#imputation on missing value with 0
#car_insurance = car_insurance.fillna(0)

#imputation the missing value with the mean
car_insurance['annual_mileage'] = car_insurance['annual_mileage'].fillna(car_insurance['annual_mileage'].mean())
car_insurance['credit_score'] = car_insurance['credit_score'].fillna(car_insurance['credit_score'].mean())

#drop missing values
#car_insurance = car_insurance.dropna()


In [5]:
#Explor the dataset
print('head:\n')
print(car_insurance.head())
print('info:\n')
print(car_insurance.info())
print('describe:\n')
print(car_insurance.describe())
print('Find missing values:')
print(car_insurance.isnull().sum())

head:

       id  age  gender driving_experience    education         income  \
0  569520    3       0               0-9y  high school    upper class   
1  750365    0       1               0-9y         none        poverty   
2  199901    0       0               0-9y  high school  working class   
3  478866    0       1               0-9y   university  working class   
4  731664    1       1             10-19y         none  working class   

   credit_score  vehicle_ownership vehicle_year  married  children  \
0      0.629027                1.0   after 2015      0.0       1.0   
1      0.357757                0.0  before 2015      0.0       0.0   
2      0.493146                1.0  before 2015      0.0       0.0   
3      0.206013                1.0  before 2015      0.0       1.0   
4      0.388366                1.0  before 2015      0.0       0.0   

   postal_code  annual_mileage vehicle_type  speeding_violations  duis  \
0        10238         12000.0        sedan                

In [7]:
#prepare the model
#create an empty list to store the models
models_list = []

#make new df with no 'outcome' and 'id' columns from car_insurance
car_insurance_features = car_insurance.drop(['outcome','id'], axis=1)

#loop throuch every possible features from col
for col in car_insurance_features:
  #fit each col in the logistic regression model
  model = logit(f"outcome~{col}", data=car_insurance).fit()
  #append the model to the list
  models_list.append(model)


Optimization terminated successfully.
         Current function value: 0.511794
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.615951
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.467092
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.603742
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.531499
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.572557
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.552412
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.572668
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.586659
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.595431
  

In [11]:
#Measurring performance
#make empty list to store accuracy
accuracies_list = []
confusion_matrix_list = []
#loop for to get the accuracy from each models in the models_list
for model in models_list:
  #get confusion matrix
  confusion_matrix = model.pred_table()
  #calculate accuracy
  accuracy = (confusion_matrix[0][0] + confusion_matrix[1][1]) / np.sum(confusion_matrix)
  #append the accuracy
  accuracies_list.append(accuracy)
  #append the confusion matrix list
  confusion_matrix_list.append(confusion_matrix)

print(accuracies_list)
print(confusion_matrix_list)

[0.7747, 0.6867, 0.7771, 0.6867, 0.7425, 0.7054, 0.7351, 0.6867, 0.6867, 0.6867, 0.6867, 0.6904, 0.6867, 0.6867, 0.6867, 0.6867]
[array([[6299.,  568.],
       [1685., 1448.]]), array([[6867.,    0.],
       [3133.,    0.]]), array([[5554., 1313.],
       [ 916., 2217.]]), array([[6867.,    0.],
       [3133.,    0.]]), array([[6239.,  628.],
       [1947., 1186.]]), array([[6321.,  546.],
       [2400.,  733.]]), array([[5594., 1273.],
       [1376., 1757.]]), array([[6867.,    0.],
       [3133.,    0.]]), array([[6867.,    0.],
       [3133.,    0.]]), array([[6867.,    0.],
       [3133.,    0.]]), array([[6867.,    0.],
       [3133.,    0.]]), array([[6801.,   66.],
       [3030.,  103.]]), array([[6867.,    0.],
       [3133.,    0.]]), array([[6867.,    0.],
       [3133.,    0.]]), array([[6867.,    0.],
       [3133.,    0.]]), array([[6867.,    0.],
       [3133.,    0.]])]


In [16]:
#Find the best performance
best_feature = car_insurance_features.columns[np.argmax(accuracies_list)]

#save best feature
best_feature_df = pd.DataFrame({'best_feature': [best_feature], 'best_accuracy': [np.max(accuracies_list)]})

print(best_feature_df)

#Find the confusion matrix of best feature
best_confusion_matrix = confusion_matrix_list[np.argmax(accuracies_list)]

#print(best_confusion_matrix)

best_acc = (best_confusion_matrix[0][0] + best_confusion_matrix[1][1]) / np.sum(best_confusion_matrix)
best_precision = best_confusion_matrix[1][1] / (best_confusion_matrix[1][1] + best_confusion_matrix[0][1])
best_recall = best_confusion_matrix[1][1] / (best_confusion_matrix[1][1] + best_confusion_matrix[1][0])
best_f1 = 2 * (best_precision * best_recall) / (best_precision + best_recall)

print(f"\nAccuracy: {best_acc}")
print(f"Precision: {best_precision}")
print(f"Recall: {best_recall}")
print(f"F1 Score: {best_f1}")

         best_feature  best_accuracy
0  driving_experience         0.7771

Accuracy: 0.7771
Precision: 0.6280453257790368
Recall: 0.7076284711139483
F1 Score: 0.6654660063034669
